In [125]:
import chart_studio.plotly as py
import plotly.graph_objs as go
import pandas as pd

In [126]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot

In [127]:
init_notebook_mode(connected=True)

In [128]:
df = pd.read_csv('1976-2018-senate.csv', encoding = 'unicode_escape')

In [129]:
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,...,party,writein,mode,candidatevotes,totalvotes,unofficial,version,Vote_Percent,candidate_scale,Vote_Percent_abs
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,republican,False,total,321236,741210,False,20171011.0,-0.433394,-1,0.433394
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,independent,False,total,1565,741210,False,20171011.0,0.002111,1,0.002111
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,democrat,False,total,400334,741210,False,20171011.0,0.540109,1,0.540109
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,libertarian,False,total,7310,741210,False,20171011.0,0.009862,1,0.009862
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,...,independent,False,total,10765,741210,False,20171011.0,0.014524,1,0.014524


In [390]:
data_slider = []

for year in df['year'].unique():    
    y =  df[(df['year']== year)]
    df_segmented = y[(y['special'] == False )]
    df_segmented=df_segmented.loc[df_segmented.groupby('state_po')['Vote_Percent_abs'].idxmax()]
    filler_highest_value = df_segmented['Vote_Percent_abs'].max()
            
            
    for i in range(len(df_segmented)):
        if df_segmented['party'].values[i] == 'independent':
            df_segmented['Vote_Percent'].values[i] = filler_highest_value+.001
        
    data_each_yr = dict(
                        type='choropleth',
                        locations = df_segmented['state_po'],
                        z=df_segmented['Vote_Percent'],
                        locationmode='USA-states',
                        text = df_segmented['candidate'],
                        marker = dict(line = dict(color = 'rgb(12,12,12)', width=1.5)),
                        zauto=False,
                        zmax = filler_highest_value+.001,
                        colorscale = [[0, '#8b0000'], 
               [0.15, '#ffcccb'],
               [0.375, '#c71585'],
               [0.5, '#FFFFFF'],
               [0.625, '#6666ff'],
               [0.85, '#add8e6'],         
               [0.9999, '#00008B'],
               [1, '#008000']])


    data_slider.append(data_each_yr)

In [391]:
steps = []

for i in range(len(data_slider)):
    step = dict(method='restyle',
                args=['visible', [False] * len(data_slider)],
                label='{}'.format(i*2 + 1976))
    step['args'][1][i] = True
    steps.append(step)
    title = '{} Senate Map'.format(i*2 + 1976)

In [392]:
sliders = [dict(active=0, pad={"t": 1}, steps=steps)]

layout = dict(title = title, geo=dict(scope='usa', showlakes = True,lakecolor='rgb(85,173,240)'
            ),
              sliders=sliders)

In [395]:
choromap = go.Figure(data = data_slider,layout = layout)

choromap.update_traces(showscale=False)

iplot(choromap)